In [2]:
import wafel
import numpy as np
import copy
import itertools
from random import *

In [ ]:
VERSION = 'us'

In [3]:
def set_inputs(g, inputs):
    g.write('gControllerPads[0].button', inputs.buttons)
    g.write('gControllerPads[0].stick_x', inputs.stick_x)
    g.write('gControllerPads[0].stick_y', inputs.stick_y)

In [4]:
#Convert from [0, 255] to [-128, 127]
def to_actual_coord(wafel_coord):
    return wafel_coord - 256*(wafel_coord > 127)

def increment_coord(coord):
    #coord = coord - 128
    coord = to_actual_coord(coord)
    if coord == -8:
        coord = 0
    elif coord == 0:
        coord = 8
    elif coord >= 127:
        coord = 127
    else:
        coord = coord + 1
    #return coord + 128
    return coord % 256

#def increment_coord(coord):
#    coord -= 128
#    if coord == -8:
#        coord = 0
#    elif coord == 0:
#        coord = 8
#    elif coord >= 127:
#        coord = 127
#    else:
#        coord = coord + 1
#    return coord + 128

In [5]:
def check_globtime_syncs_mod64(filename, start_frame, end_frame,
                               vars_to_sync_at_end):
    game = wafel.Game('sm64_{0}.dll'.format(VERSION))
    m64 = wafel.load_m64(filename)
    for f in range(start_frame):
        set_inputs(game, m64[1][f])
        game.advance()
    start_state = game.save_state()
    start_globtime = game.read('gGlobalTimer')
    for f in range(start_frame, end_frame):
        set_inputs(game, m64[1][f])
        game.advance()
    var_values = {}
    for var in vars_to_sync_at_end:
        var_values[var] = game.read(var)
    #final_starcount = game.read('gMarioState.numStars')
    successes = []
    for globtime_offset in range(64):
        game.load_state(start_state)
        game.write('gGlobalTimer', start_globtime + globtime_offset)
        for f in range(start_frame, end_frame):
            set_inputs(game, m64[1][f])
            game.advance()
        succeeded = True
        for var in vars_to_sync_at_end:
            if var_values[var] != game.read(var):
                succeeded = False
        if succeeded:
            print(globtime_offset)
            successes.append(globtime_offset)
    return successes

In [54]:
def detect_a_presses(filename):
    m64 = wafel.load_m64(filename)
    for f in range(len(m64[1])):
        if m64[1][f].buttons >= 0x8000:
            print(f)

In [6]:
#For all frames with a walking action and low speed, determine
#whether it's possible to change whether Mario dusts without
#changing his position on the next frame. For the sake of RNG manip.
#Exhaustively checks all (x,y) pairs.

def identify_dustable_frames(filename, start_frame, end_frame):
    game = wafel.Game('sm64_{0}.dll'.format(VERSION))
    m64 = wafel.load_m64(filename)
    for f in range(start_frame):
        set_inputs(game, m64[1][f])
        game.advance()
    rng_changes = []
    for f in range(start_frame, end_frame):
        prev_state = game.save_state()
        set_inputs(game, m64[1][f])
        game.advance()
        if game.read('gMarioState.action') == 67109952 and game.read('gMarioState.forwardVel') < 17:
            rng_reached = game.read('gRandomSeed16')
            x_reached = game.read('gMarioState.pos[0]')
            z_reached = game.read('gMarioState.pos[2]')
            found_soln = None
            for x in range(256):
                for y in range(256):
                    game.load_state(prev_state)
                    set_inputs(game, wafel.Input(buttons = m64[1][f].buttons, stick_x = x, stick_y = y))
                    game.advance()
                    if (game.read('gRandomSeed16') != rng_reached
                        and game.read('gMarioState.pos[0]') == x_reached
                        and game.read('gMarioState.pos[2]') == z_reached):
                        found_soln = (x, y)
            game.load_state(prev_state)
            set_inputs(game, m64[1][f])
            game.advance()
            if found_soln != None:
                print('Found an alternate dust ' + str(f))
                rng_changes.append((f, m64[1][f].stick_x, m64[1][f].stick_y, found_soln[0], found_soln[1]))
            else:
                print('Found no alternate dust for ' + str(f))
    return rng_changes

In [7]:
#For all frames with a walking action and low speed, determine
#whether it's possible to change whether Mario dusts without
#changing his position on the next frame. For the sake of RNG manip.
#Only checks a subset of (x,y) pairs that should give similar angles.
#Uses actual trig instead of in-game trig table, so I can't guarantee
#that it will find all possible dustable frames. There's a window that I widened
#to a few AU to try to improve the chances that it captures all of them.
#Of course, this slows it down, but it's still much faster than the other one
def identify_dustable_frames_faster(filename, start_frame, end_frame):
    
    #get angles of all possible (x,y) joystick pairs
    #hacky way to do this, but should be fine
    #using actual arctan2 instead of sm64's trig
    #tables, so will need to check a wider range of
    #possible values.
    sticks = np.meshgrid(np.arange(0, 256, 1), np.arange(0, 256, 1))
    sticks = np.array([sticks[1], sticks[0]])
    raw_sticks = copy.deepcopy(sticks)
    #convert raw joystick inputs to processed stick inputs
    sticks[sticks > 127] -= 256
    for coord in range(2):
        sticks[coord][np.abs(sticks[coord]) < 8] = 0
        sticks[coord][sticks[coord] >= 8] -= 6
        sticks[coord][sticks[coord] <= -8] += 6
    angles = np.arctan2(sticks[0], sticks[1])
    
    game = wafel.Game('sm64_{0}.dll'.format(VERSION))
    m64 = wafel.load_m64(filename)
    for f in range(start_frame):
        set_inputs(game, m64[1][f])
        game.advance()
    rng_changes = []
    for f in range(start_frame, end_frame):
        prev_state = game.save_state()
        set_inputs(game, m64[1][f])
        game.advance()
        if game.read('gMarioState.action') == 67109952 and game.read('gMarioState.forwardVel') < 17:
            angle = angles[m64[1][f].stick_x, m64[1][f].stick_y]
            
            
            rng_reached = game.read('gRandomSeed16')
            x_reached = game.read('gMarioState.pos[0]')
            z_reached = game.read('gMarioState.pos[2]')
            found_soln = None
            
            
            #check for positions with angle within about 2 HAU (extra slack just in case due to
            #not using sm64 trig tables)
            similar_angle_joys = np.where(np.abs(angles - angle) < .0091)
            for attempt in range(similar_angle_joys[0].shape[0]):
                game.load_state(prev_state)
                try_x = raw_sticks[0][similar_angle_joys[0][attempt], similar_angle_joys[1][attempt]]
                try_y = raw_sticks[1][similar_angle_joys[0][attempt], similar_angle_joys[1][attempt]]

                set_inputs(game, wafel.Input(buttons = m64[1][f].buttons, stick_x = try_x, stick_y = try_y))
                game.advance()
                if (game.read('gRandomSeed16') != rng_reached
                    and game.read('gMarioState.pos[0]') == x_reached
                    and game.read('gMarioState.pos[2]') == z_reached):
                    found_soln = (try_x, try_y)
                    break            
            
            game.load_state(prev_state)
            set_inputs(game, m64[1][f])
            game.advance()
            if found_soln != None:
                print('Found an alternate dust ' + str(f))
                rng_changes.append((f, m64[1][f].stick_x, m64[1][f].stick_y, found_soln[0], found_soln[1]))
            else:
                print('Found no alternate dust for ' + str(f))
    return rng_changes

In [8]:
#rng behavior based on Ukikipedia pseudocode
def rng_advance():
    global rngseed
    if rngseed == 22026:
        rngseed = np.uint16(0)
    
    A = np.uint16(rngseed << 8)
    A = A ^ rngseed
    
    rngseed = A
    left_byte = np.uint16(A << 8)
    right_byte = A >> 8
    rngseed = left_byte + right_byte
    
    A = np.uint16(A << 8) >> 7
    A = A ^ rngseed
    B = A >> 1
    B = B ^ np.uint16(65408)
    
    if A % 2 == 0:
        if B == 43605:
            rngseed = 0
        else:
            rngseed = B ^ np.uint16(8180)
    else:
        rngseed = B ^ np.uint16(33152)
    return rngseed

def get_rng_array():
    global rngseed
    rngseed = np.uint16(0)
    values = []
    while rngseed > 0 or len(values) == 0:
        values.append(rngseed)
        rng_advance()
    return values

In [9]:
def check_rng_syncs(filename, start_frame, end_frame,
                               vars_to_sync_at_end,
                    low = 0, high = 65114):
    game = wafel.Game('sm64_{0}.dll'.format(VERSION))
    m64 = wafel.load_m64(filename)
    for f in range(start_frame):
        set_inputs(game, m64[1][f])
        game.advance()
    start_state = game.save_state()
    start_globtime = game.read('gGlobalTimer')
    for f in range(start_frame, end_frame):
        set_inputs(game, m64[1][f])
        game.advance()
    var_values = {}
    for var in vars_to_sync_at_end:
        var_values[var] = game.read(var)
    successes = []
    game.write('gRandomSeed16', 0)
    print(low)
    print(high)
    for ind, val in enumerate(get_rng_array()[low:high]):
        game.load_state(start_state)
        game.write('gRandomSeed16', int(val))
        for f in range(start_frame, end_frame):
            set_inputs(game, m64[1][f])
            game.advance()
        succeeded = True
        for var in vars_to_sync_at_end:
            if var_values[var] != game.read(var):
                succeeded = False
        if succeeded:
            print(ind+low)
            successes.append(ind+low)
    return successes

In [43]:
def evaluate_fitness(game, m64, start_state,
                     start_frame, end_frame,
                     possible_alternatives,
                     goal_rng_ind, possible_alternatives_choices,
                     desired_x, desired_z,
                     rng_val_to_ind_map):
    game.load_state(start_state)
    cur_alternative_ind = 0
    for f in range(start_frame, end_frame):
        if cur_alternative_ind < len(possible_alternatives) and f == possible_alternatives[cur_alternative_ind][0]:
            if possible_alternatives_choices[cur_alternative_ind] == 1:
                x, y = possible_alternatives[cur_alternative_ind][3:]
                set_inputs(game, wafel.Input(m64[1][f].buttons, x, y))
            else:
                set_inputs(game, m64[1][f])
            cur_alternative_ind += 1
        else:
            set_inputs(game, m64[1][f])
        game.advance()
    rng_result = rng_val_to_ind_map[game.read('gRandomSeed16')]
    cost = abs(rng_result - goal_rng_ind)
    if cost > 33000:
        cost = abs(cost - 65114)
    if desired_x != game.read('gMarioState.pos[0]') or desired_z != game.read('gMarioState.pos[2]'):
        cost += 10000000
    if (rng_result - goal_rng_ind) % 4 != 0:
        cost += 12
    return cost
    
        

#Tries to manipulate dust from start_frame to end_frame
#to achieve a certain RNG index at end_frame. Checks
#Mario's position to verify it didn't desync.
def target_rng(filename, filename_output, start_frame, end_frame, goal_rng_ind,
               max_attempts = 100000):
    game = wafel.Game('sm64_{0}.dll'.format(VERSION))
    m64 = wafel.load_m64(filename)
    for f in range(start_frame):
        set_inputs(game, m64[1][f])
        game.advance()
    start_state = game.save_state()
    
    #RNG array: mapping from indices to values. Interested in the reverse
    rng_array = get_rng_array()
    rng_val_to_ind_map = {rng_array[ind]: ind for ind in range(65114)}
    
    for f in range(start_frame, end_frame):
        set_inputs(game, m64[1][f])
        game.advance()
        
    desired_x = game.read('gMarioState.pos[0]')
    desired_z = game.read('gMarioState.pos[2]')
    game.load_state(start_state)
    
    possible_alternatives = identify_dustable_frames_faster(filename, start_frame, end_frame)
    print('Identified {0} frames on which one can do RNG manip'.format(len(possible_alternatives)))
    
    cur_choices = np.zeros(len(possible_alternatives))
    cur_fitness = evaluate_fitness(game, m64, start_state,
                                    start_frame, end_frame,
                                    possible_alternatives,
                                    goal_rng_ind, cur_choices,
                                    desired_x, desired_z,
                                    rng_val_to_ind_map)
    change_prob = .3
    temp = 3
    soln = None
    for attempts in range(max_attempts):
        if attempts % 100 == 0:
            print(attempts)
            print(cur_fitness)
            temp *= .98
            if change_prob*len(possible_alternatives) > 3:
                change_prob *= .98
        to_change = np.random.choice(a=[0, 1], size=(len(possible_alternatives)), p=[1 - change_prob, change_prob])
        #Sometimes only want to change close to the end
        if random() < .3:
            to_change[0:int(random()*len(possible_alternatives))] = 0
        #print(cur_choices)
        #print(to_change)
        new_choices = (cur_choices + to_change) % 2
        #print(new_choices)
        new_fitness = evaluate_fitness(game, m64, start_state,
                                       start_frame, end_frame,
                                       possible_alternatives,
                                       goal_rng_ind, new_choices,
                                       desired_x, desired_z,
                                       rng_val_to_ind_map)
        #print(new_fitness)
        if new_fitness == 0:
            soln = new_choices
            print('Found an RNG resync solution.')
            print(game.read('gRandomSeed16'))
            break
        if new_fitness < cur_fitness or random() < np.exp(-(new_fitness - cur_fitness) / temp):
            cur_choices = new_choices
            cur_fitness = new_fitness
        #print(cur_choices)
    if soln is None:
        print('Could not find a solution. Final fitness is {0}'.format(new_fitness))
        return
    for choice_ind in range(len(possible_alternatives)):
        if soln[choice_ind] == 1:
            alt = possible_alternatives[choice_ind]
            m64[1][alt[0]] = wafel.Input(m64[1][alt[0]].buttons, alt[3], alt[4])
    
    wafel.save_m64(filename_output, m64[0], m64[1])

In [ ]:
#Does an exhaustive search of all possible RNG manipulations starting at
#start_frame to try to match the given variables at end_frame.
#At the moment, doesn't do anything to store the results
#Checks Mario's position to verify the sync, and also may stop early if
#any variables in check_vars desync.
def find_working_indices_with_manip(filename, start_frame, end_frame, check_vars = [], custom_checkpoints_to_add = [],
                                    inds_to_check = range(65114), consecutive_reduction = False, num_files_to_save = 10,
                                    hack_index = None):
    game = wafel.Game('sm64_{0}.dll'.format(VERSION))
    m64 = wafel.load_m64(filename)
    for f in range(start_frame):
        set_inputs(game, m64[1][f])
        game.advance()
    
    #RNG array: mapping from indices to values. Interested in the reverse
    rng_array = get_rng_array()
    rng_val_to_ind_map = {rng_array[ind]: ind for ind in range(65114)}
    if hack_index != None:
        game.write('gRandomSeed16', int(rng_array[hack_index]))
    start_state = game.save_state()
    
    num_files_saved = 0
    
    
    check_points = np.linspace(start_frame, end_frame, 10)
    check_points = set([int(f) for f in check_points] + custom_checkpoints_to_add)
    correct_vals_map = dict()
    for f in range(start_frame, end_frame):
        set_inputs(game, m64[1][f])
        if f in check_points:
            correct_vals_map[f] = dict()
            for var in check_vars:
                correct_vals_map[f][var] = game.read(var)
        game.advance()
    print(correct_vals_map)
        
    desired_x = game.read('gMarioState.pos[0]')
    desired_z = game.read('gMarioState.pos[2]')
    game.load_state(start_state)
    
    possible_alternatives = identify_dustable_frames_faster(filename, start_frame, end_frame)
    print('Identified {0} frames on which one can do RNG manip'.format(len(possible_alternatives)))
    
    #we might have situations where there are like 6 consecutive frames to manip
    #instead of checking all 64, let's just try yes*k and then no*(6-k) for all k
    to_check_possibilities = []
    if consecutive_reduction:
        for possible_alternatives_choices in itertools.product([0, 1], repeat = len(possible_alternatives)):
            skip = False
            for i in range(len(possible_alternatives_choices)):
                if i > 0 and (possible_alternatives_choices[i] == 1 and
                              possible_alternatives_choices[i - 1] == 0 and
                              possible_alternatives[i][0] == possible_alternatives[i - 1][0] + 1):
                    skip = True
                    break
            if not skip:
                to_check_possibilities.append(possible_alternatives_choices)
    
    working = []
    
    #all_possibilities = list(itertools.product([0, 1], repeat = len(possible_alternatives)))
    for count, rng_ind in enumerate(inds_to_check):
        failed_prefixes = set()
        #default iterator: exhaustive
        choices_iterator = itertools.product([0, 1], repeat = len(possible_alternatives))
        if consecutive_reduction:
            choices_iterator = to_check_possibilities
        for possible_alternatives_choices in choices_iterator:        
            if count % 100 == 0 and max(possible_alternatives_choices) == 0:
                print(count)
            skip = False
            for i in range(len(possible_alternatives_choices)):
                if tuple(possible_alternatives_choices[:i]) in failed_prefixes:
                    skip = True
                    break
            if skip:
                continue
            game.load_state(start_state)
            game.write('gRandomSeed16', int(rng_array[rng_ind]))
            
            cur_alternative_ind = 0
            succeeded = True
            for f in range(start_frame, end_frame):
                #if f == 17582:
                #    print('Past bomps: ' + str(rng_ind))
                if f in check_points:
                    for var in check_vars:
                        if correct_vals_map[f][var] != game.read(var):
                            failed_prefixes.add(tuple(possible_alternatives_choices[:cur_alternative_ind]))
                            succeeded = False
                            break
                if not succeeded:
                    break
                if cur_alternative_ind < len(possible_alternatives) and f == possible_alternatives[cur_alternative_ind][0]:
                    if possible_alternatives_choices[cur_alternative_ind] == 1:
                        x, y = possible_alternatives[cur_alternative_ind][3:]
                        set_inputs(game, wafel.Input(m64[1][f].buttons, x, y))
                    else:
                        set_inputs(game, m64[1][f])
                    cur_alternative_ind += 1
                else:
                    set_inputs(game, m64[1][f])
                game.advance()
            if desired_x != game.read('gMarioState.pos[0]') or desired_z != game.read('gMarioState.pos[2]'):
                succeeded = False
            if succeeded:
                print(str(possible_alternatives) + ' ' + str(rng_ind))
                working.append(rng_ind)
                if num_files_saved < num_files_to_save:
                    num_files_saved += 1
                    m64_modify = wafel.load_m64(filename)
                    for choice_ind in range(len(possible_alternatives)):
                        if possible_alternatives_choices[choice_ind] == 1:
                            alt = possible_alternatives[choice_ind]
                            m64_modify[1][alt[0]] = wafel.Input(m64_modify[1][alt[0]].buttons, alt[3], alt[4])
                    wafel.save_m64('generated_m64s/hack_index_{0}_frame_{1}.m64'.format(rng_ind, start_frame + 1), m64_modify[0], m64_modify[1])
                break
                
    return working

In [11]:
def inspect(filename, frame_number_start, frame_number_end, extra_vars = []):
    game = wafel.Game('sm64_{0}.dll'.format(VERSION))
    m64 = wafel.load_m64(filename)
    for f in range(frame_number_start):
        set_inputs(game, m64[1][f])
        game.advance()
    for f in range(frame_number_start, frame_number_end):
        print('Frame number: {0}'.format(f))
        print('Mario position: {0}'.format(game.read('gMarioState.pos')))
        print('Mario yaw: {0}'.format(game.read('gMarioState.faceAngle[1]')))
        print('Mario hspd: {0}'.format(game.read('gMarioState.forwardVel')))
        for extra_var in extra_vars:
            print('{0}: {1}'.format(extra_var, game.read(extra_var)))
        set_inputs(game, m64[1][f])
        game.advance()

In [12]:
def locate_framenums_close(filename, mario_x, mario_y, mario_z, radius = 100):
    game = wafel.Game('sm64_{0}.dll'.format(VERSION))
    m64 = wafel.load_m64(filename)
    for f in range(len(m64[1])):
        mario_pos = game.read('gMarioState.pos')
        dist = ((mario_pos[0] - mario_x)**2 + (mario_pos[1] - mario_y)**2 + (mario_pos[2] - mario_z)**2)**.5
        if dist < radius:
            print('{0}: {1} pos, {2} stars'.format(f, mario_pos, game.read('gMarioState.numStars')))
        set_inputs(game, m64[1][f])
        game.advance()

In [ ]:
def hex_in_improvement(filename_old, filename_improved, filename_output,
                       frames_back_to_search_slowwalks = 5000):
    game = wafel.Game('sm64_{0}.dll'.format(VERSION))
    start_state = game.save_state()
    m64_old = wafel.load_m64(filename_old)
    m64_improved = wafel.load_m64(filename_improved)
    print(len(m64_old[1]))
    print(len(m64_improved[1]))
    
    rng_array = get_rng_array()
    rng_array_inv = {rng_array[i]: i for i in range(len(rng_array))}
    
    #collect relevant information about the TASes
    #First, identify which frame we should sync on by finding last frame before Mario stops
    #moving. Measure position and number of stars on this frame; the new TAS should match this.
    prev_mario_pos, goal_mario_pos, num_stars, frame_to_sync_on_improved, actual_index = None, None, None, None, None
    for f in range(len(m64_improved[1])):
        set_inputs(game, m64_improved[1][f])
        game.advance()
        new_mario_pos = game.read('gMarioState.pos')
        if new_mario_pos != prev_mario_pos:
            num_stars = game.read('gMarioState.numStars')
            frame_to_sync_on_improved = f
            goal_mario_pos = new_mario_pos
            actual_index = rng_array_inv[game.read('gRandomSeed16')]
        prev_mario_pos = new_mario_pos
        
    print(num_stars)
       
    
    
    game.load_state(start_state)
    frame_to_sync_on_old, orig_rng_index = None, None
    for f in range(len(m64_old[1])):
        set_inputs(game, m64_old[1][f])
        game.advance()
        if game.read('gMarioState.pos') == goal_mario_pos and game.read('gMarioState.numStars') == num_stars:
            frame_to_sync_on_old = f
            orig_rng_index = rng_array_inv[game.read('gRandomSeed16')]
            break
    
    print('Try to synchronize after {0} stars with pos {1} on frame {2}'.format(num_stars, goal_mario_pos, frame_to_sync_on_improved))
            
    print('Want to reach rng index {0}'.format(orig_rng_index))
    print('Actually observed index: {0}'.format(actual_index))
    hexed_inputs_wrongrng = m64_improved[1][:frame_to_sync_on_improved] + m64_old[1][frame_to_sync_on_old:]
    filename_intermediate = filename_output[:-4] + '_with_ending_hexed_badrng.m64'
    wafel.save_m64(filename_intermediate, m64_improved[0], hexed_inputs_wrongrng)
    
    target_rng(filename_intermediate, filename_output,
               frame_to_sync_on_improved - frames_back_to_search_slowwalks, frame_to_sync_on_improved,
               orig_rng_index)

In [2]:
def aim_for_target(filename, start_frame, end_frame, vars_list, var_targets,
                       low = 0, high = 65114, repair_mario_positions = False):
        #Try all possible rng_indices at start_frame and at end_frame evaluate
        #how close the variables are to the targets.
        #by default, taxicab dist
        game = wafel.Game('sm64_{0}.dll'.format(VERSION))
        m64 = wafel.load_m64(filename)
        mario_positions = []
        for f in range(start_frame):
            set_inputs(game, m64[1][f])
            game.advance()
        start_state = game.save_state()
        if repair_mario_positions:
            for f in range(start_frame, end_frame):
                set_inputs(game, m64[1][f])
                game.advance()
                mario_positions.append(game.read('gMarioState.pos'))
            error = 0
            for i in range(len(vars_list)):
                actual = game.read(vars_list[i])
                #print(actual)
                error += abs(actual - var_targets[i])
            #print(error)
        #print(mario_positions)
        distances = []
        rng_array = get_rng_array()
        for rng_ind in range(low, high):
            rng_val = int(rng_array[rng_ind])
            if rng_ind % 1000 == 0 and rng_ind > low:
                print(rng_ind)
                print(min(distances))
            game.load_state(start_state)
            game.write('gRandomSeed16', rng_val)
            for f in range(start_frame, end_frame):
                set_inputs(game, m64[1][f])
                game.advance()
                #if repair_mario_positions:
                #    game.write('gMarioState.pos[0]', mario_positions[f - start_frame][0])
                #    game.write('gMarioState.pos[1]', mario_positions[f - start_frame][1])
                #    game.write('gMarioState.pos[2]', mario_positions[f - start_frame][2])
            error = 0
            for i in range(len(vars_list)):
                actual = game.read(vars_list[i])
                #print(actual)
                error += abs(actual - var_targets[i])
            distances.append(error)
            #print(error)
        return distances